# Data Club Prep

This notebook is just a scratch place where I can do counts and calculates for my data club presentation.

In [2]:
# %load ../start.py
# Load useful extensions

# Activate the autoreload extension for easy reloading of external packages
%reload_ext autoreload
%autoreload 2

# Trun on the water mark
%reload_ext watermark
%watermark -u -d -g

# Load ipycache extension
%reload_ext ipycache
from ipycache import CacheMagics
CacheMagics.cachedir = '../cachedir'

# Add project library to path
import sys
sys.path.insert(0, '../../lib/python')

# The usual suspects
import os
import numpy as np
import pandas as pd

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_context('poster')

# Turn off scientific notation
np.set_printoptions(precision=5, suppress=True)


last updated: 2017-10-10 
Git hash: 8c83e87b7c4eac097d2ea2f50eee0e3a81393eaa


In [4]:
# %load ../../bin/load.py
from pymongo import MongoClient
with open('/home/fearjm/Projects/ncbi_remap/output/.mongodb_host', 'r') as fh:
    host = fh.read().strip()
client = MongoClient(host=host, port=27022)
db = client['sra2']
remap = db['remap']
ncbi = db['ncbi']

In [7]:
# How many SRR can't be downloaded
list(remap.aggregate([
    {'$unwind': '$runs'},
    {
        '$match': {
            'runs.pre_aln_flags': 'download_bad'
        }
    },
    {
        '$count': 'cnt'
    }
]))

[{'cnt': 72}]

In [8]:
# How many SRRs have a problem with their quality score
list(remap.aggregate([
    {'$unwind': '$runs'},
    {
        '$match': {
            'runs.pre_aln_flags': 'quality_scores_bad'
        }
    },
    {
        '$count': 'cnt'
    }
]))

[{'cnt': 4}]

In [23]:
# How many good samples are there?
libsize = pd.read_pickle('../../output/libsize_downstream_analysis.pkl')
libsize[~libsize.flag_low_libsize & ~libsize.flag_short_read_len].shape

(19083, 4)

In [22]:
# Which samples did I keep only one pair
list(remap.aggregate([
    {'$unwind': '$runs'},
    {
        '$match': {
            '$or': [
                {'runs.pre_aln_flags': 'keep_R1'},
                {'runs.pre_aln_flags': 'keep_R2'},
            ]
        }
    },
    {'$count': 'cnt'}
]))

[{'cnt': 714}]

In [24]:
!ls ../../output/*.pkl

../../output/contamination_downstream_analysis.pkl
../../output/correlation_downstream_analysis.pkl
../../output/libsize_downstream_analysis.pkl
../../output/mapability_downstream.pkl
../../output/mapability_downstream_analysis.pkl
../../output/strandedness_downstream_analysis.pkl


In [25]:
strand = pd.read_pickle('../../output/strandedness_downstream_analysis.pkl')

In [27]:
strand.indicator_strand.value_counts()

 0    13846
-2     3949
 2     2117
-1     1675
 1      603
Name: indicator_strand, dtype: int64